In [20]:
import pandas as pd

In [21]:
# newest = pd.read_csv("fastapi-server/newest_data.csv")
forecasted_data = pd.read_csv("fastapi-server/forecasted_pollutant.csv")
aqi_7_days_lag_df = pd.read_csv("fastapi-server/aqi_7_days_lag.csv")
aqi_14_days_lag_df = pd.read_csv("fastapi-server/aqi_14_days_lag.csv")

In [22]:
forecasted_data.head()


,Unnamed: 0,Aqi,Carbon_monoxide,District,Dust,Location_id,Nitrogen_dioxide,Ozone,Pm_10,Pm_25,Sulphur_dioxide
0,2024-10-04 00:00:00,112.15275,659.62530,Khushab,36.618546,1.0,15.318667,55.985096,78.476260,63.799767,8.939727
1,2024-10-04 01:00:00,110.65491,626.06610,Khushab,41.261627,1.0,13.231259,62.902004,79.606190,67.186170,10.122407
2,2024-10-04 02:00:00,109.95273,599.77905,Khushab,47.167793,1.0,10.713866,75.119064,79.480420,70.001780,11.268904
3,2024-10-04 03:00:00,109.29117,570.00110,Khushab,53.182262,1.0,7.900428,92.052960,79.712190,72.456910,11.993094
4,2024-10-04 04:00:00,106.89256,526.43384,Khushab,58.695183,1.0,5.883153,108.875830,80.343056,72.917160,12.293127


In [23]:
# newest.tail()

In [24]:
forecasted_data.head()
# renaming the unnamed: 0 column to date
forecasted_data.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
forecasted_data.head()



,date,Aqi,Carbon_monoxide,District,Dust,Location_id,Nitrogen_dioxide,Ozone,Pm_10,Pm_25,Sulphur_dioxide
0,2024-10-04 00:00:00,112.15275,659.62530,Khushab,36.618546,1.0,15.318667,55.985096,78.476260,63.799767,8.939727
1,2024-10-04 01:00:00,110.65491,626.06610,Khushab,41.261627,1.0,13.231259,62.902004,79.606190,67.186170,10.122407
2,2024-10-04 02:00:00,109.95273,599.77905,Khushab,47.167793,1.0,10.713866,75.119064,79.480420,70.001780,11.268904
3,2024-10-04 03:00:00,109.29117,570.00110,Khushab,53.182262,1.0,7.900428,92.052960,79.712190,72.456910,11.993094
4,2024-10-04 04:00:00,106.89256,526.43384,Khushab,58.695183,1.0,5.883153,108.875830,80.343056,72.917160,12.293127


In [25]:
# filtering both data to have only the date, District and Aqi columns
# newest_filtered = newest[['date', 'District', 'Aqi']]
forecasted_data_filtered = forecasted_data[['date', 'District', 'Aqi']]

# concatenating the filtered dataframes
combined_data = forecasted_data_filtered
combined_data.head()


,date,District,Aqi
0,2024-10-04 00:00:00,Khushab,112.15275
1,2024-10-04 01:00:00,Khushab,110.65491
2,2024-10-04 02:00:00,Khushab,109.95273
3,2024-10-04 03:00:00,Khushab,109.29117
4,2024-10-04 04:00:00,Khushab,106.89256


In [26]:
combined_data

,date,District,Aqi
0,2024-10-04 00:00:00,Khushab,112.152750
1,2024-10-04 01:00:00,Khushab,110.654910
2,2024-10-04 02:00:00,Khushab,109.952730
3,2024-10-04 03:00:00,Khushab,109.291170
4,2024-10-04 04:00:00,Khushab,106.892560
...,...,...,...
647995,2025-01-01 19:00:00,Mianwali,60.760490
647996,2025-01-01 20:00:00,Mianwali,61.532925
647997,2025-01-01 21:00:00,Mianwali,62.956880
647998,2025-01-01 22:00:00,Mianwali,64.446686


In [27]:
# converting the date column to datetime
combined_data['date'] = pd.to_datetime(combined_data['date'])

combined_data.head()

C:\Users\timmy_3aupohg\AppData\Local\Temp\ipykernel_32040\1197255911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['date'] = pd.to_datetime(combined_data['date'])


,date,District,Aqi
0,2024-10-04 00:00:00,Khushab,112.15275
1,2024-10-04 01:00:00,Khushab,110.65491
2,2024-10-04 02:00:00,Khushab,109.95273
3,2024-10-04 03:00:00,Khushab,109.29117
4,2024-10-04 04:00:00,Khushab,106.89256


In [28]:
# converting the data to daily by selecting the max in the day for that day
# this should be done for each district
# Group by 'district' and 'date', then select the max value for each group
daily_max = combined_data.groupby(['District', combined_data['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max.columns = ['District', 'date', 'Aqi']

daily_max.head()


,District,date,Aqi
0,Attock,2024-10-04,148.294680
1,Attock,2024-10-05,89.888330
2,Attock,2024-10-06,79.794080
3,Attock,2024-10-07,82.654915
4,Attock,2024-10-08,82.598130


In [29]:
daily_max.shape

(3240, 3)

In [30]:
# saving the daily_max dataframe to a csv file
daily_max.to_csv("daily_historical_data.csv", index=False)

In [31]:
# creating a daily max version for the forecasted data
# selecting the columns to work with
forecasted_data_filtered = forecasted_data[['date', 'District', 'Aqi']]

# converting the date column to datetime
forecasted_data_filtered['date'] = pd.to_datetime(forecasted_data_filtered['date'])

# converting the data to daily by selecting the max in the day for that day
# this should be done for each district
daily_max_forecasted = forecasted_data_filtered.groupby(['District', forecasted_data_filtered['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max_forecasted.columns = ['District', 'date', 'Aqi']

daily_max_forecasted.head()

C:\Users\timmy_3aupohg\AppData\Local\Temp\ipykernel_32040\2306417717.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecasted_data_filtered['date'] = pd.to_datetime(forecasted_data_filtered['date'])


,District,date,Aqi
0,Attock,2024-10-04,148.294680
1,Attock,2024-10-05,89.888330
2,Attock,2024-10-06,79.794080
3,Attock,2024-10-07,82.654915
4,Attock,2024-10-08,82.598130


In [32]:
# checking the shape of the daily_max_forecasted dataframe
daily_max_forecasted.shape

(3240, 3)

In [33]:
# saving the daily_max_forecasted dataframe to a csv file
daily_max_forecasted.to_csv("daily_forecasted_data.csv", index=False)

In [34]:
# creating a daily max version for the 7 days lag data
# selecting the columns to work with
aqi_7_days_lag_df = aqi_7_days_lag_df[['date', 'district', 'Aqi']]

# converting the date column to datetime
aqi_7_days_lag_df['date'] = pd.to_datetime(aqi_7_days_lag_df['date'])

# converting the data to daily by selecting the max in the day for that day 
daily_max_7_days_lag = aqi_7_days_lag_df.groupby(['district', aqi_7_days_lag_df['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max_7_days_lag.columns = ['district', 'date', 'Aqi']

daily_max_7_days_lag.head()
# saving the daily_max_7_days_lag dataframe to a csv file
daily_max_7_days_lag.to_csv("daily_7_days_lag_data.csv", index=False)
daily_max_7_days_lag = aqi_7_days_lag_df.groupby(['district', aqi_7_days_lag_df['date'].dt.date])['Aqi'].max().reset_index()



In [35]:
daily_max_7_days_lag

,district,date,Aqi
0,Attock,2024-09-27,111.361770
1,Attock,2024-09-28,104.757130
2,Attock,2024-09-29,76.539910
3,Attock,2024-09-30,71.301796
4,Attock,2024-10-01,71.201990
...,...,...,...
3487,Vehari,2024-12-28,68.179570
3488,Vehari,2024-12-29,68.179565
3489,Vehari,2024-12-30,68.179570
3490,Vehari,2024-12-31,68.179570


In [36]:
# doing the same for the 14 days lag data
# selecting the columns to work with
aqi_14_days_lag_df = aqi_14_days_lag_df[['date', 'district', 'Aqi']]

# converting the date column to datetime
aqi_14_days_lag_df['date'] = pd.to_datetime(aqi_14_days_lag_df['date'])

# converting the data to daily by selecting the max in the day for that day 
daily_max_14_days_lag = aqi_14_days_lag_df.groupby(['district', aqi_14_days_lag_df['date'].dt.date])['Aqi'].max().reset_index() 

# Rename columns for clarity
daily_max_14_days_lag.columns = ['district', 'date', 'Aqi']

daily_max_14_days_lag.head()
# saving the daily_max_14_days_lag dataframe to a csv file
daily_max_14_days_lag.to_csv("daily_14_days_lag_data.csv", index=False)


In [37]:
daily_max_14_days_lag

,district,date,Aqi
0,Attock,2024-09-20,80.576260
1,Attock,2024-09-21,64.520226
2,Attock,2024-09-22,64.974120
3,Attock,2024-09-23,65.133860
4,Attock,2024-09-24,65.196810
...,...,...,...
3739,Vehari,2024-12-28,131.148320
3740,Vehari,2024-12-29,131.142180
3741,Vehari,2024-12-30,131.038310
3742,Vehari,2024-12-31,131.061720
